#### This file contains the sentiment analysis model that will classify events as ```success``` or ```failure```

We will train our sentiment analysis model in three ways and compare their performance

## Sklearn Implementation

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
sentiment140_sklearn = pd.read_parquet('../data-processed/sentiment140_model_data.parquet')

In [17]:
sentiment140_sklearn.head()

,text,label
0,"[switchfoot, httptwitpiccom2y1zl, awww, that, ...",0
1,"[upset, cant, updat, facebook, text, it, might...",0
2,"[kenichan, dive, mani, time, ball, manag, save...",0
3,"[whole, bodi, feel, itchi, like, fire]",0
4,"[nationwideclass, no, behav, all, im, mad, her...",0


In [36]:
sentiment140_sklearn = sentiment140_sklearn.dropna(axis='rows')

Prepare embeddings from text to train the model

In [37]:
vectorizer = TfidfVectorizer()
X = sentiment140_sklearn['text']
y = sentiment140_sklearn['label']
X

0          [switchfoot, httptwitpiccom2y1zl, awww, that, ...
1          [upset, cant, updat, facebook, text, it, might...
2          [kenichan, dive, mani, time, ball, manag, save...
3                     [whole, bodi, feel, itchi, like, fire]
4          [nationwideclass, no, behav, all, im, mad, her...
                                 ...                        
1599995                 [woke, up, school, best, feel, ever]
1599996    [thewdbcom, cool, hear, old, walt, interview, ...
1599997                   [readi, mojo, makeov, ask, detail]
1599998    [happi, 38th, birthday, boo, alll, time, tupac...
1599999    [happi, charitytuesday, thenspcc, sparkschar, ...
Name: text, Length: 1599999, dtype: object

In [38]:
X = vectorizer.fit_transform(X)

In [39]:
X

<1599999x783601 sparse matrix of type '<class 'numpy.float64'>'
	with 12204586 stored elements in Compressed Sparse Row format>

In [40]:
random_state = 42

In [41]:
Xtr, Xts, ytr, yts = train_test_split(
    X,
    y,
    test_size=0.3, 
    random_state=random_state
)

In [51]:
model_sklearn = LogisticRegression(random_state=random_state, max_iter=100, penalty='l2').fit(Xtr.astype(int), ytr)

/Users/priyangshupal/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [53]:
mean_squared_error(yts, model_sklearn.predict(Xts.astype(int)))

0.49532291666666667